In [1]:
from tqdm.auto import tqdm 
import pandas as pd
import numpy as np
import xarray as xr
import glob
%matplotlib inline

import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [2]:
datapath="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/Data/ECMWF/ERA5_25kmx3hr/"
path="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/besttracks/wp/"
output="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/create_ts/outputs/"
target="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/create_ts/outputs/targets/"

In [3]:
track = sorted(glob.glob(path+'wp_2015.csv'))

In [4]:
tracksDF = pd.read_csv(track[0])

In [5]:
tracksDF['name'].unique()
stormnames = list(tracksDF['name'].unique())

In [6]:
[str(tracksDF[tracksDF['name']==stormnames[0]].time[i]).split(':')[0] for i in range(len(tracksDF[tracksDF['name']==stormnames[0]].time))]

['2015-01-12',
 '2015-01-12',
 '2015-01-12',
 '2015-01-12',
 '2015-01-12',
 '2015-01-12',
 '2015-01-13',
 '2015-01-13',
 '2015-01-13',
 '2015-01-13',
 '2015-01-13',
 '2015-01-13',
 '2015-01-13',
 '2015-01-13',
 '2015-01-14',
 '2015-01-14',
 '2015-01-14',
 '2015-01-14',
 '2015-01-14',
 '2015-01-14',
 '2015-01-14',
 '2015-01-14',
 '2015-01-15',
 '2015-01-15',
 '2015-01-15',
 '2015-01-15',
 '2015-01-15',
 '2015-01-15',
 '2015-01-15',
 '2015-01-15',
 '2015-01-16',
 '2015-01-16',
 '2015-01-16',
 '2015-01-16',
 '2015-01-16',
 '2015-01-16',
 '2015-01-16',
 '2015-01-16',
 '2015-01-17',
 '2015-01-17',
 '2015-01-17',
 '2015-01-17',
 '2015-01-17',
 '2015-01-17',
 '2015-01-17',
 '2015-01-17',
 '2015-01-18',
 '2015-01-18',
 '2015-01-18',
 '2015-01-18',
 '2015-01-18',
 '2015-01-18',
 '2015-01-18',
 '2015-01-18',
 '2015-01-19',
 '2015-01-19',
 '2015-01-19',
 '2015-01-19',
 '2015-01-19',
 '2015-01-19',
 '2015-01-19',
 '2015-01-19',
 '2015-01-20',
 '2015-01-20',
 '2015-01-20',
 '2015-01-20',
 '2015-01-

In [115]:
#1000_200, 1000_300, 1000_500, 1000_700, 1000_850, 850_200, 850_250, 850_300, 850_500, 925_200, 925_250

In [7]:
#ssingle level variables
def output_indices(TCtrack=None,ERA5date=None,ERA5hour=None):
    allindices = []
    for timeidx in range(len(TCtrack)):#len(track['time'])):
        datetrack,hourtrack = TCtrack['time'][timeidx].split(':')[0],TCtrack['time'][timeidx].split(':')[1][0:2]
        ####################################################################################################
        # Find the indices in ERA5 data with the same date as track
        ####################################################################################################
        dateind = []
        for ind,obj in enumerate(ERA5date):
            if obj==datetrack:
                dateind.append(ind)
        del ind,obj
        hourind = []
        hourextract = ERA5hour[int(np.min(np.asarray(dateind))):int(np.max(np.asarray(dateind)))+1]
        for ind,obj in enumerate(hourextract):
            if obj==hourtrack:            
                hourind.append(ind)
        allindices.append((int(np.min(np.asarray(dateind))),int(hourind[0])))
    return allindices

def extract_var(dataset=None,var='var138',indices=None):
    extractedvar = []
    for i in (range(len(indices))):
        realindex = indices[i][0]+indices[i][1]
        extractedvar.append(dataset[var][int(realindex),...].data)
    return np.asarray(extractedvar)

def largearea(dataset=None,invar=None,indices=None):
    if len(invar.shape) != 3:
        invar = np.squeeze(invar)
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices)))),
    attrs=dict(description="coords with matrices"),)
    
    var_out=np.zeros((len(indices),64,64))
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_orad[it,:]
        try:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
        except:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:64,0:64]
    return var_out

def largearea_withpres(dataset=None,invar=None,indices=None):
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","plev","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices))),
               plev=(["plev"],dataset.plev.data)),
    attrs=dict(description="coords with matrices"),)
    var_out=np.zeros((len(indices),len(dm2.plev.data),64,64))
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_orad[it,:]
        for ip in range(len(dm2.plev.data)):
            try:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
            except:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:64,0:64]
    return var_out 

In [8]:
###Shear vars
def createmask(dm=None,irad=None,orad=None,lonselect=None,latselect=None):
    mask = []
    for ti in range(len(tc_irad)):
        lonselect = dm.lon.sel(lon=slice(orad[ti,:][2],orad[ti,:][3])).data
        latselect = np.flipud(dm.lat.sel(lat=slice(orad[ti,:][1],orad[ti,:][0])).data)
        if (lonselect.shape != 64) or (latselect.shape != 64):
            lon2d,lat2d = np.meshgrid(lonselect[0:64],latselect[0:64])
        else:
            lon2d,lat2d = np.meshgrid(lonselect,latselect)
        #############################################################################################
        latcriteria = np.logical_and(lat2d>irad[ti][0],lat2d<irad[ti][1])
        loncriteria = np.logical_and(lon2d>irad[ti][2],lon2d<irad[ti][3])
        allcriteria = np.logical_and(loncriteria,latcriteria)
        mask.append(allcriteria)
    return mask

In [9]:
def readyear(year=None,sheartype=None):
    dm2 = xr.open_dataset(datapath+'/shear/shear_'+str(sheartype)+'_'+str(year)+'.nc')
    #tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/besttracks/wp/*_'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,era5_date,era5_hour

In [10]:
TCshears = {}
for shear in ['1000_200', '1000_300', '1000_500', '1000_700', '1000_850', \
              '850_200', '850_250', '850_300', '850_500', '925_200', '925_250']:
    dm2,era5_date,era5_hour = readyear(2015,shear)
    TCtempstore = []
    for TCobj in tqdm(stormnames):
        track=tracksDF[tracksDF['name']==TCobj].reset_index()
        lon1=track['lon'].to_numpy()
        lat1=track['lat'].to_numpy()
        lonx=np.mod(lon1,360)
        pos = arr = np.stack((lat1, lonx), axis=1)
        ###########################################################################
        indices_store = output_indices(track,era5_date,era5_hour)
        ###########################################################################
        mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
        ###########################################################################
        tc_irad=np.empty((len(indices_store),4))
        tc_irad[:,0] = pos[:,0]-2
        tc_irad[:,1] = pos[:,0]+2
        tc_irad[:,2] = pos[:,1]-2
        tc_irad[:,3] = pos[:,1]+2
        
        tc_orad=np.empty((len(indices_store),4))
        tc_orad[:,0] = pos[:,0]-8
        tc_orad[:,1] = pos[:,0]+8
        tc_orad[:,2] = pos[:,1]-8
        tc_orad[:,3] = pos[:,1]+8
        ###########################################################################
        smallsvarout = [largearea(dm2,mysvar[i],indices_store) for i in (range(len(mysvar)))]
        
        svarname1 = ['shear_'+shear]#['shear_850_200']
        svarname = ['shear']
        
        svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
        lonselect = dm2.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
        latselect = np.flipud(dm2.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
        lon2d,lat2d = np.meshgrid(lonselect,latselect)
        
        mymask = createmask(dm=dm2,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
        #############################################################################################
        tsdict = {}
        for ind,obj in (enumerate(svarname)):
            tslist = [svardict[svarname[ind]][i,...][~mymask[i]] for i in range(len(mymask))]
            tsdict[svarname1[ind]] = [np.nanmean(obj) for obj in tslist]
        #############################################################################################
        TCtempstore.append(tsdict)
    TCshears[shear] = TCtempstore
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

...

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

In [11]:
smallTCshear1 = {}
smallTCshear2 = {}
smallTCshear3 = {}
smallTCshear4 = {}
smallTCshear5 = {}
smallTCshear6 = {}
smallTCshear7 = {}
smallTCshear8 = {}
smallTCshear9 = {}
smallTCshear10 = {}
smallTCshear11 = {}
smallTCshear12 = {}
smallTCshear13 = {}
smallTCshear14 = {}
smallTCshear15 = {}
smallTCshear16 = {}
smallTCshear17 = {}
smallTCshear18 = {}
#smallTCshear19 = {}
#smallTCshear20 = {}
#smallTCshear21 = {}
#smallTCshear22 = {}
#smallTCshear23 = {}
#smallTCshear24 = {}

for shear in ['1000_200', '1000_300', '1000_500', '1000_700', '1000_850', \
              '850_200', '850_250', '850_300', '850_500', '925_200', '925_250']:
    smallTCshear1['shear_'+str(shear)] = (TCshears[shear][0]['shear_'+str(shear)])
    smallTCshear2['shear_'+str(shear)] = (TCshears[shear][1]['shear_'+str(shear)])
    smallTCshear3['shear_'+str(shear)] = (TCshears[shear][2]['shear_'+str(shear)])
    smallTCshear4['shear_'+str(shear)] = (TCshears[shear][3]['shear_'+str(shear)])
    smallTCshear5['shear_'+str(shear)] = (TCshears[shear][4]['shear_'+str(shear)])
    smallTCshear6['shear_'+str(shear)] = (TCshears[shear][5]['shear_'+str(shear)])
    smallTCshear7['shear_'+str(shear)] = (TCshears[shear][6]['shear_'+str(shear)])
    smallTCshear8['shear_'+str(shear)] = (TCshears[shear][7]['shear_'+str(shear)])
    smallTCshear9['shear_'+str(shear)] = (TCshears[shear][8]['shear_'+str(shear)])
    smallTCshear10['shear_'+str(shear)] = (TCshears[shear][9]['shear_'+str(shear)])
    smallTCshear11['shear_'+str(shear)] = (TCshears[shear][10]['shear_'+str(shear)])
    smallTCshear12['shear_'+str(shear)] = (TCshears[shear][11]['shear_'+str(shear)])
    smallTCshear13['shear_'+str(shear)] = (TCshears[shear][12]['shear_'+str(shear)])
    smallTCshear14['shear_'+str(shear)] = (TCshears[shear][13]['shear_'+str(shear)])
    smallTCshear15['shear_'+str(shear)] = (TCshears[shear][14]['shear_'+str(shear)])
    smallTCshear16['shear_'+str(shear)] = (TCshears[shear][15]['shear_'+str(shear)])
    smallTCshear17['shear_'+str(shear)] = (TCshears[shear][16]['shear_'+str(shear)])
    smallTCshear18['shear_'+str(shear)] = (TCshears[shear][17]['shear_'+str(shear)])
    #smallTCshear19['shear_'+str(shear)] = (TCshears[shear][18]['shear_'+str(shear)])
    #smallTCshear20['shear_'+str(shear)] = (TCshears[shear][19]['shear_'+str(shear)])
    #smallTCshear21['shear_'+str(shear)] = (TCshears[shear][20]['shear_'+str(shear)])
    #smallTCshear22['shear_'+str(shear)] = (TCshears[shear][21]['shear_'+str(shear)])
    #smallTCshear23['shear_'+str(shear)] = (TCshears[shear][22]['shear_'+str(shear)])
    #smallTCshear24['shear_'+str(shear)] = (TCshears[shear][23]['shear_'+str(shear)])
   

In [12]:
shears1=pd.DataFrame.from_dict(smallTCshear1)
shears2=pd.DataFrame.from_dict(smallTCshear2)
shears3=pd.DataFrame.from_dict(smallTCshear3)
shears4=pd.DataFrame.from_dict(smallTCshear4)
shears5=pd.DataFrame.from_dict(smallTCshear5)
shears6=pd.DataFrame.from_dict(smallTCshear6)
shears7=pd.DataFrame.from_dict(smallTCshear7)
shears8=pd.DataFrame.from_dict(smallTCshear8)
shears9=pd.DataFrame.from_dict(smallTCshear9)
shears10=pd.DataFrame.from_dict(smallTCshear10)
shears11=pd.DataFrame.from_dict(smallTCshear11)
shears12=pd.DataFrame.from_dict(smallTCshear12)
shears13=pd.DataFrame.from_dict(smallTCshear13)
shears14=pd.DataFrame.from_dict(smallTCshear14)
shears15=pd.DataFrame.from_dict(smallTCshear15)
shears16=pd.DataFrame.from_dict(smallTCshear16)
shears17=pd.DataFrame.from_dict(smallTCshear17)
shears18=pd.DataFrame.from_dict(smallTCshear18)
#shears19=pd.DataFrame.from_dict(smallTCshear19)
#shears20=pd.DataFrame.from_dict(smallTCshear20)
#shears21=pd.DataFrame.from_dict(smallTCshear21)
#shears22=pd.DataFrame.from_dict(smallTCshear22)
#shears23=pd.DataFrame.from_dict(smallTCshear23)
#shears24=pd.DataFrame.from_dict(smallTCshear24)



In [13]:
stormnames

['MEKKHALA',
 'BAVI',
 'NOUL',
 'DOLPHIN',
 'CHAN-HOM',
 'LINFA',
 'NANGKA',
 'HALOLA',
 'SOUDELOR',
 'MOLAVE',
 'GONI',
 'ATSANI',
 'KILO',
 'KROVANH',
 'DUJUAN',
 'CHOI-WAN',
 'CHAMPI',
 'MELOR']

In [14]:
shears1.to_csv(output+'shears_wpouter_2015_MEKKHALA.csv')
shears2.to_csv(output+'shears_wpouter_2015_BAVI.csv')
shears3.to_csv(output+'shears_wpouter_2015_NOUL.csv')
shears4.to_csv(output+'shears_wpouter_2015_DOLPHIN.csv')
shears5.to_csv(output+'shears_wpouter_2015_CHAN-HOM.csv')
shears6.to_csv(output+'shears_wpouter_2015_LINFA.csv')
shears7.to_csv(output+'shears_wpouter_2015_NANGKA.csv')
shears8.to_csv(output+'shears_wpouter_2015_HALOLA.csv')
shears9.to_csv(output+'shears_wpouter_2015_SOUDELOR.csv')
shears10.to_csv(output+'shears_wpouter_2015_MOLAVE.csv')
shears11.to_csv(output+'shears_wpouter_2015_GONI.csv')
shears12.to_csv(output+'shears_wpouter_2015_ATSANI.csv')
shears13.to_csv(output+'shears_wpouter_2015_KILO.csv')
shears14.to_csv(output+'shears_wpouter_2015_KROVANH.csv')
shears15.to_csv(output+'shears_wpouter_2015_DUJUAN.csv')
shears16.to_csv(output+'shears_wpouter_2015_CHOI-WAN.csv')
shears17.to_csv(output+'shears_wpouter_2015_CHAMPI.csv')
shears18.to_csv(output+'shears_wpouter_2015_MELOR.csv')
#shears19.to_csv(output+'shears_wpouter_2015_MANGKHUT.csv')
#shears20.to_csv(output+'shears_wpouter_2015_TRAMI.csv')
#shears21.to_csv(output+'shears_wpouter_2015_USAGI.csv')
#shears22.to_csv(output+'shears_wpouter_2015_MAN-YI.csv')
#shears23.to_csv(output+'shears_wpouter_2015_TALAS.csv')
#shears24.to_csv(output+'shears_wpouter_2015_NORU.csv')



script ends here..!!!

Divergence

In [120]:
dm2 = xr.open_dataset(datapath+'/div/div_2021.nc')

In [121]:
#Pressure vars - divergence3.nc')
TCdiv_ts = []
for TCobj in tqdm(stormnames):
    track=tracksDF[tracksDF['name']==TCobj].reset_index()
    lat1=track['lat'].to_numpy()
    lon1=track['lon'].to_numpy()
    lonx=np.mod(lon1,360)
    pos = arr = np.stack((lat1, lonx), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallpvarout = [largearea_withpres(dm2,mypvar[i],indices_store) for i in (range(len(mypvar)))]    
    pvarname1 = ['outdiv']
    pvarname = ['var155']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    lonselect = dm2.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm2.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    #latselect = dm2.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data
    lon2d,lat2d = np.meshgrid(lonselect,latselect)
    mymask = createmask(dm=dm2,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm2.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...][~mymask[i]] for i in range(len(mymask))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname1[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCdiv_ts.append(ts_pdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm2]

  0%|          | 0/7 [00:00<?, ?it/s]

In [122]:
plv = [str(int(obj)) for obj in dm2.plev.data/100]

In [123]:
def create_pandadict(divdict=None,divlv=plv):
    tempdivdict = {'outdiv_'+divlv[i]:divdict['outdiv'][:,i] for i in range(divdict['outdiv'].shape[1])}
    alldict = {**tempdivdict}
    return alldict

In [124]:
storeTCdicts = {}
for ind in range(len(TCdiv_ts)):
    storeTCdicts[stormnames[ind]] = pd.DataFrame.from_dict(create_pandadict(divdict=TCdiv_ts[ind]))

In [125]:
for ind,obj in enumerate(stormnames):
    storeTCdicts[obj].to_csv(output+'2021_outerdiv_wpac_'+str(obj)+'.csv')

Eq. potential temperature

In [130]:
def readyear(year=None):
    dm2 = xr.open_mfdataset([
                             datapath+'/eqt/eqt_200_'+str(year)+'.nc',datapath+'/eqt/eqt_250_'+str(year)+'.nc',\
                             datapath+'/eqt/eqt_300_'+str(year)+'.nc',datapath+'/eqt/eqt_400_'+str(year)+'.nc',\
                             datapath+'/eqt/eqt_500_'+str(year)+'.nc',datapath+'/eqt/eqt_600_'+str(year)+'.nc',\
                             datapath+'/eqt/eqt_700_'+str(year)+'.nc',datapath+'/eqt/eqt_800_'+str(year)+'.nc',\
                             datapath+'/eqt/eqt_850_'+str(year)+'.nc',datapath+'/eqt/eqt_1000_'+str(year)+'.nc',\
                             datapath+'/eqt/eqt_925_'+str(year)+'.nc'])
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,era5_date,era5_hour

dm2,era5_date,era5_hour = readyear(2021)

In [131]:
TCeqt_ts = []
for TCobj in tqdm(stormnames):
    track=tracksDF[tracksDF['name']==TCobj].reset_index()
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallsvarout = [largearea(dm2,mysvar[i],indices_store) for i in (range(len(mysvar)))]
    svarname1 = ['outeqt1000','outeqt200','outeqt250','outeqt300','outeqt400',\
                 'outeqt500','outeqt600','outeqt700','outeqt800',\
                 'outeqt850','outeqt925']
   
    svarname = ['eqt1000','eqt200','eqt250','eqt300','eqt400',\
                'eqt500','eqt600','eqt700','eqt800',\
                'eqt850','eqt925']
    
    
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    lonselect = dm2.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm2.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)

    mymask = createmask(dm=dm2,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...][~mymask[i]] for i in range(len(mymask))]
        tsdict[svarname1[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCeqt_ts.append(tsdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/7 [00:00<?, ?it/s]

In [132]:
storeTCdicts = {}
for ind in range(len(TCeqt_ts)):
    storeTCdicts[stormnames[ind]] = pd.DataFrame.from_dict(TCeqt_ts[ind])

In [133]:
for ind,obj in enumerate(stormnames):
    storeTCdicts[obj].to_csv(output+'2021_outereqt_wpac_'+str(obj)+'.csv')

script ends here..!!!